# STEP 1 : Merge SWOT files along cycles for each pass

### Ingvild Olden Bjerkelund

## Information about the script : 

- This is Step 1, which merges all cycles into one file, for each pass. 
- Does not add/remove anything from the files. 
- This is based on the original structure of how I downloaded the files, where I kept track of which "day" the passes encountered (following the fixed track of the satellite). Would have to change the input structure to fit changes.
- Each file is first downloaded from the HiTide webpage : https://hitide.podaac.earthdatacloud.nasa.gov/ 
- The files are NetCDF format.
    - Open NetCDF-files : xr.open_dataset(file_name)
    - Save NetCDF-files : file_name.to_netcdf(path + '/' + file_name + '.nc')
- Initially the files keep a name, where the information about pass, cycle, start and end date, etc.
- When merging alon the cycles, this information is lost, and then we need to first get the wanted information, and then make it as a "file_name_dimension". To keep track of the dates and so on. 
- The webpage also allows for merging files already before downloading. However, then the information about the passes is lost. And you can not add new cycles later on, then you need to find all cycles and merge again.
- This script allows for a second merge, when new cycles arrive. 


### Import all packages

In [6]:
# Import python packages
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy as cy
from   matplotlib.dates import DateFormatter
import cartopy.crs as ccrs 
import cmocean
import netCDF4 as nc
import glob as glob
import pandas as pd

### Paths for input and output data : 

In [2]:
#main_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_0_RAW_DATA'   #'c:\\Users\ingvi\Desktop\MASTER_DATA\SWOT_DATA'
main_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_0_RAW_DATA_v2'
#save_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_1_RAW_MERGED' #'C:\\Users\ingvi\Desktop\MASTER_DATA\SWOT_DATA_MERGED'
save_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_1_RAW_MERGED/version_2' 

### Structure of files (Change if needed) : 

In [3]:
# List of passes per day : 
day_01 = ['pass_003', 'pass_005', 'pass_007', 'pass_008', 'pass_009', 'pass_010', 'pass_012', 'pass_014', 'pass_016', 'pass_018']
day_02 = ['pass_031', 'pass_033', 'pass_035', 'pass_036', 'pass_037', 'pass_038', 'pass_040', 'pass_042', 'pass_044', 'pass_046']
day_03 = ['pass_059', 'pass_061', 'pass_063', 'pass_064', 'pass_065', 'pass_066', 'pass_068', 'pass_070', 'pass_072', 'pass_074']
day_04 = ['pass_087', 'pass_089', 'pass_091', 'pass_092', 'pass_093', 'pass_094', 'pass_096', 'pass_098', 'pass_100']
day_05 = ['pass_113', 'pass_115', 'pass_117', 'pass_119', 'pass_120', 'pass_121', 'pass_122', 'pass_124', 'pass_126', 'pass_128']
day_06 = ['pass_141', 'pass_143', 'pass_145', 'pass_147', 'pass_148', 'pass_149', 'pass_150', 'pass_152', 'pass_154', 'pass_156']
day_07 = ['pass_169', 'pass_171', 'pass_173', 'pass_175', 'pass_176', 'pass_177', 'pass_178', 'pass_180', 'pass_182', 'pass_184']
day_08 = ['pass_197', 'pass_199', 'pass_201', 'pass_203', 'pass_204', 'pass_205', 'pass_206', 'pass_208', 'pass_210', 'pass_212']
day_09 = ['pass_225', 'pass_227', 'pass_229', 'pass_231', 'pass_232', 'pass_233', 'pass_234', 'pass_236', 'pass_238', 'pass_240']
day_10 = ['pass_253', 'pass_255', 'pass_257', 'pass_259', 'pass_260', 'pass_262', 'pass_264', 'pass_266', 'pass_268']
day_11 = ['pass_281', 'pass_283', 'pass_285', 'pass_286', 'pass_287', 'pass_288', 'pass_290', 'pass_292', 'pass_294', 'pass_296']
day_12 = ['pass_309', 'pass_311', 'pass_313', 'pass_314', 'pass_315', 'pass_316', 'pass_318', 'pass_320', 'pass_322', 'pass_324']
day_13 = ['pass_337', 'pass_339', 'pass_341', 'pass_342', 'pass_343', 'pass_344', 'pass_346', 'pass_348', 'pass_350', 'pass_352']
day_14 = ['pass_365', 'pass_367', 'pass_369', 'pass_370', 'pass_371', 'pass_372', 'pass_374', 'pass_376', 'pass_378', 'pass_391']
day_15 = ['pass_393', 'pass_395', 'pass_397', 'pass_398', 'pass_399', 'pass_400', 'pass_402', 'pass_404', 'pass_406', 'pass_419']
day_16 = ['pass_421', 'pass_423', 'pass_425', 'pass_426', 'pass_427', 'pass_428', 'pass_430', 'pass_432', 'pass_434', 'pass_447']
day_17 = ['pass_449', 'pass_451', 'pass_453', 'pass_454', 'pass_455', 'pass_456', 'pass_458', 'pass_460', 'pass_462', 'pass_475']
day_18 = ['pass_477', 'pass_479', 'pass_481', 'pass_482', 'pass_483', 'pass_484', 'pass_486', 'pass_488', 'pass_490', 'pass_503']
day_19 = ['pass_505', 'pass_507', 'pass_509', 'pass_510', 'pass_511', 'pass_512', 'pass_514', 'pass_516', 'pass_518', 'pass_531']
day_20 = ['pass_533', 'pass_535', 'pass_536', 'pass_537', 'pass_538', 'pass_539', 'pass_540', 'pass_542', 'pass_544', 'pass_546', 'pass_559']
day_21 = ['pass_561', 'pass_563', 'pass_564', 'pass_565', 'pass_566', 'pass_568', 'pass_570', 'pass_572', 'pass_574']

# List of days : 
days_nm = ['day_01', 'day_02', 'day_03', 'day_04', 'day_05', 'day_06', 'day_07', 'day_08', 'day_09', 'day_10', 'day_11', 'day_12', 'day_13',
           'day_14', 'day_15', 'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21']
days_df = [day_01, day_02, day_03, day_04, day_05, day_06, day_07, day_08, day_09, day_10, day_11, day_12, day_13, day_14, day_15, day_16, 
           day_17, day_18, day_19, day_20, day_21]

# List of days, divided in 2 :
days_nm1 = ['day_01', 'day_02', 'day_03', 'day_04', 'day_05', 'day_06', 'day_07', 'day_08', 'day_09', 'day_10']
days_df1 = [day_01, day_02, day_03, day_04, day_05, day_06, day_07, day_08, day_09, day_10]

days_nm2 = ['day_11', 'day_12', 'day_13', 'day_14', 'day_15', 'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21']
days_df2 = [day_11, day_12, day_13, day_14, day_15, day_16, day_17, day_18, day_19, day_20, day_21]

### Function to Concat and save wanted information in a new dimension :

In [ ]:
''' FUNCTION TO CONCATENATE THE FILES : '''
def concat_files(main,day,passname):
    # Keep track of the pass and day :
    print('Working on ' + passname + ' on ' + day)
    # The path to the files :
    path = main + '/' + day + '/' + passname
    print(path)
    # The files in the path :
    files = glob.glob(path+'/*')
    files.sort()
    # Initialize the new dimension name :
    filenamedim = []
    # The number of files to concatenate :
    len_files = len(files)
    for i in range(len(files)):
        #print(files[i])
        # The cycle of the file : 
        #print('Working on ' + 'cyc_' + files[i][108:111])  # CHANGE THIS IF NEEDED !!! Old : files[i][83:86])
        print('Working on ' + 'cyc_' + files[i][111:114])
        # The Start time of the file :
        #print('Working on ' + 'time_' + files[i][116:124]) # CHANGE THIS IF NEEDED !!! Old : files[i][91:99])
        print('Working on ' + 'time_' + files[i][119:127])
        # The new dimension name :
        # CHANGE THIS IF NEEDED !!!
        filenamedim.append('cyc_' + files[i][111:114] + '_' + 'time_' + files[i][119:127]) 
    # Concatenating the files : (meanings that the files are concatenated along the new dimension)
    concated_files = xr.concat([xr.open_dataset(files[a]) for a in range(len_files)], dim =filenamedim) 
    return concated_files

#### Test the naming structure

In [ ]:
paths = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_0_RAW_DATA_v2/day_01/pass_003/SWOT_L2_LR_SSH_Expert_002_003_20230811T040145_20230811T045314_PGC0_01_subsetted.nc4'
print(paths[111:114])
print(paths[119:127])
#print('cyc_' + paths[108:111] + '_' + 'time_' + paths[116:124])
print('cyc_' + paths[111:114] + '_' + 'time_' + paths[119:127])

### For-loop to loop trough the files (with given structure) :

In [ ]:
''' CONCATENATING THE FILES IN A FOR-LOOP : '''
# Looping over the days :
for days in days_nm:
    # Looping over the passes in the days :
    for pass_nm in days_df[days_nm.index(days)]:
        # Concatenating the files :
        concated_files = concat_files(main_path,days,pass_nm)
        # The save path :
        save_path = save_path
        # Saving the file :
        concated_files.to_netcdf(save_path + '\\' + pass_nm + '.nc')

### Open and check the new merged-files : 

In [4]:
# Open pass_003.nc, pass_005.nc, pass_007.nc, pass_008.nc
path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_1_RAW_MERGED/version_2/pass_003.nc'
pass_003 = xr.open_dataset(path)
pass_003['ssh_karin'].concat_dim

<xarray.DataArray 'concat_dim' (concat_dim: 27)> Size: 2kB
array(['cyc_002_time_20230811', 'cyc_003_time_20230901',
       'cyc_004_time_20230921', 'cyc_005_time_20231012',
       'cyc_006_time_20231102', 'cyc_007_time_20231123',
       'cyc_008_time_20231214', 'cyc_009_time_20240104',
       'cyc_010_time_20240125', 'cyc_011_time_20240214',
       'cyc_012_time_20240306', 'cyc_013_time_20240327',
       'cyc_014_time_20240417', 'cyc_015_time_20240508',
       'cyc_016_time_20240529', 'cyc_017_time_20240619',
       'cyc_018_time_20240710', 'cyc_019_time_20240730',
       'cyc_020_time_20240820', 'cyc_021_time_20240910',
       'cyc_022_time_20241001', 'cyc_023_time_20241022',
       'cyc_024_time_20241112', 'cyc_025_time_20241203',
       'cyc_026_time_20241223', 'cyc_027_time_20250113',
       'cyc_028_time_20250203'], dtype='<U21')
Coordinates:
  * concat_dim  (concat_dim) <U21 2kB 'cyc_002_time_20230811' ... 'cyc_028_ti...

In [5]:
pass_003.isel(concat_dim=0)

<xarray.Dataset> Size: 84MB
Dimensions:                                (num_lines: 2062, num_pixels: 69,
                                            num_sides: 2)
Coordinates:
    latitude                               (num_lines, num_pixels) float64 1MB ...
    longitude                              (num_lines, num_pixels) float64 1MB ...
    latitude_nadir                         (num_lines) float64 16kB ...
    longitude_nadir                        (num_lines) float64 16kB ...
    concat_dim                             <U21 84B 'cyc_002_time_20230811'
Dimensions without coordinates: num_lines, num_pixels, num_sides
Data variables: (12/98)
    time                                   (num_lines) datetime64[ns] 16kB ...
    time_tai                               (num_lines) datetime64[ns] 16kB ...
    ssh_karin                              (num_lines, num_pixels) float64 1MB ...
    ssh_karin_qual                         (num_lines, num_pixels) float64 1MB ...
    ssh_karin_uncert                       (num_lines, num_pixels) float64 1MB ...
    ssha_karin                             (num_lines, num_pixels) float64 1MB ...
    ...                                     ...
    swh_ssb_cor_source                     (num_lines, num_pixels) float32 569kB ...
    swh_ssb_cor_source_2                   (num_lines, num_pixels) float32 569kB ...
    wind_speed_ssb_cor_source              (num_lines, num_pixels) float32 569kB ...
    wind_speed_ssb_cor_source_2            (num_lines, num_pixels) float32 569kB ...
    volumetric_correlation                 (num_lines, num_pixels) float64 1MB ...
    volumetric_correlation_uncert          (num_lines, num_pixels) float64 1MB ...
Attributes: (12/63)
    Conventions:                                   CF-1.7
    title:                                         Level 2 Low Rate Sea Surfa...
    institution:                                   JPL
    source:                                        Ka-band radar interferometer
    history:                                       2024-01-25T11:12:55Z : Cre...
    platform:                                      SWOT
    ...                                            ...
    ellipsoid_flattening:                          0.0033528106647474805
    good_ocean_data_percent:                       67.31458661908201
    ssha_variance:                                 0.019234787072090646
    references:                                    V1.2.1
    equator_longitude:                             -25.72
    history_json:                                  [{"date_time": "2024-09-14...